In [1]:
import os
import hydra
import pytorch_lightning as pl
import torch
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative
from omegaconf import OmegaConf,open_dict
import omegaconf



In [2]:
cfg_auto_aggl = omegaconf.OmegaConf.load('config/auto_aggl.yaml')
cfg_denoiser_data = omegaconf.OmegaConf.load('config/denoiser/data.yaml')
cfg_denoiser_encode = omegaconf.OmegaConf.load('config/denoiser/encoder.yaml')
cfg_denoiser_global_config = omegaconf.OmegaConf.load('config/denoiser/global_config.yaml')
cfg_denoiser_model = omegaconf.OmegaConf.load('config/denoiser/model.yaml')

cfg_ae_vq_vae = omegaconf.OmegaConf.load('config/ae/vq_vae.yaml')
cfg_ae_global_config = omegaconf.OmegaConf.load('config/ae/global_config.yaml')
cfg_ae_model = omegaconf.OmegaConf.load('config/ae/model.yaml')


cfg = OmegaConf.merge(
    cfg_auto_aggl,
    {"denoiser": cfg_denoiser_data},
    {"denoiser": cfg_denoiser_encode},
    {"denoiser": cfg_denoiser_global_config},
    {"denoiser": cfg_denoiser_model},
    {"ae": cfg_ae_vq_vae},
    {"ae": cfg_ae_global_config},
    {"ae": cfg_ae_model},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/global_config.yaml')},
    #{"verifier": omegaconf.OmegaConf.load('config/verifier/model.yaml')},
)

In [5]:

data_home_dir = '/data/jhahn/data/shape_dataset/'
data_type_name = 'shape'

with open_dict(cfg):
    cfg.experiment_output_path = data_home_dir+'experiment_output/'
    cfg.denoiser.data.data_val_dir=f'{data_home_dir}/pc_data/{data_type_name}/val/'
    #cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/everyday_epoch100_bs64/training/last.ckpt'
    cfg.denoiser.ckpt_path= data_home_dir+f'output/denoiser/shape_epoch10/training/last.ckpt'
    cfg.inference_dir= data_home_dir+'results'
    cfg.denoiser.data.val_batch_size=1
    #cfg.verifier.ckpt_path= '/disk2/data/breaking-bad-dataset/output/verifier/everyday_epoch100_bs64/training/last.ckpt'

#print(OmegaConf.to_yaml(cfg))
#print(cfg.experiment_output_path)

In [6]:
# fix the seed
pl.seed_everything(cfg.test_seed, workers=True)

# create directories for inference outputs
inference_dir = os.path.join(cfg.experiment_output_path, "inference", cfg.inference_dir)
os.makedirs(inference_dir, exist_ok=True)

# initialize data
from jahn_src.data_loader_jhahn import build_test_dataloader


#from puzzlefusion_plusplus.denoiser.dataset.dataset import build_test_dataloader
#cfg.denoiser.data.matching_data_path= '/disk2/data/breaking-bad-dataset/data/matching_data'

test_loader = build_test_dataloader(cfg.denoiser)

Seed set to 123
100%|██████████| 10/10 [00:00<00:00, 1037.30it/s]


In [8]:
import importlib
import jahn_src.auto_agg_jhahn
importlib.reload(jahn_src.myutil)
importlib.reload(jahn_src.auto_agg_jhahn)
from jahn_src.auto_agg_jhahn import AutoAgglomerative

#import importlib
#import puzzlefusion_plusplus.auto_aggl
#importlib.reload(puzzlefusion_plusplus.auto_aggl)
#from puzzlefusion_plusplus.auto_aggl import AutoAgglomerative

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [9]:

model = AutoAgglomerative(cfg)

denoiser_weights = torch.load(cfg.denoiser.ckpt_path)['state_dict']

model.denoiser.load_state_dict(
    {k.replace('denoiser.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('denoiser.')}
)

model.encoder.load_state_dict(
    {k.replace('encoder.', ''): v for k, v in denoiser_weights.items() 
     if k.startswith('encoder.')}
)

# load verifier weights    
#verifier_weights = torch.load(cfg.verifier.ckpt_path)['state_dict']
#model.verifier.load_state_dict({k.replace('verifier.', ''): v for k, v in verifier_weights.items()})


trainer = pl.Trainer(accelerator=cfg.accelerator, max_epochs=1, logger=False)
trainer.test(model=model, dataloaders=test_loader)


/tmp/ipykernel_363996/3149461437.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  denoiser_weights = torch.load(cfg.denoiser.ckpt_path)['state_dict']
GPU available: True 

Testing: |          | 0/? [00:00<?, ?it/s]

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
['shape/vase/1/fractured_0']
gt_trans_torch.Size([1, 20, 3])_tensor([[[-3.4528e-17, -1.5099e-17,  8.8818e-18],
         [ 8.2818e-02,  2.6249e-01,  1.3594e-01],
         [ 9.5489e-03,  6.9158e-02,  3.7160e-02],
         [-6.3827e-02, -1.4085e-01, -5.1764e-02],
         [-3.3740e-03, -3.6094e-01, -2.6379e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 0.0000e+

[{'eval/part_acc': 0.1845238208770752,
  'eval/rmse_t': 10.802184104919434,
  'eval/rmse_r': 68.3436508178711,
  'eval/shape_cd': 825.6810913085938}]